In [2]:
# comment out these lines if you have already installed these modules

#!pip install nltk
#!pip install tensorflow
#!pip install keras
import sklearn as sk
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
import string
import nltk
import tensorflow
from sklearn.model_selection import train_test_split
from collections import Counter
from pandas import DataFrame
from matplotlib import pyplot
import re
from nltk.stem import WordNetLemmatizer 
from keras.preprocessing.text import Tokenizer

In C:\Users\sriku\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sriku\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sriku\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\sriku\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\sriku\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sriku\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Preprocess Code

In [4]:
def process_news(news):
    _news = news.replace('b\"', "")
    _news = _news.replace('b\'', "")
    _news = _news.lower()
    _news = re.sub("[^a-zA-Z]", " ",_news)
    _news = re.sub('[\s]+', ' ', _news)
    
    tokens = _news.split(" ")
    if "" in tokens:
        tokens.remove("")
    
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    #remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    
    _news = ' '.join(tokens)    
     
    return _news

Load data

In [5]:
def read_data():

    data = pd.read_csv("../Datasets/djia/Combined_News_DJIA.csv")
    
    dfs = []
    data["News"] = ""
    for i in range(1,25):
        col = "Top"+str(i)
        data["News"] = data["News"] +" "+ data[col]
    data = data.dropna()
    data['PreProcessedNews'] = data['News'].map(process_news)
    
    data = data[['Date', 'News', 'PreProcessedNews', 'Label']]
    
    stock_prices = "../Datasets/djia/upload_DJIA_table.csv"
    stock_data = pd.read_csv(stock_prices)
    
    print(data.head(2))
    print(stock_data.head(2))
    
    
    #merged_dataframe = data.merge(stock_data, how='inner', on='Date')
    merged_dataframe = pd.merge(data, stock_data, how='inner', on = 'Date')

    
    
    
    Xy_train = merged_dataframe[:int(len(data)*0.6)]
    Xy_valid = merged_dataframe[int(len(data)*0.6):int(len(data)*0.8)]
    Xy_test = merged_dataframe[int(len(data)*0.8):]
    
    return merged_dataframe, Xy_train, Xy_valid, Xy_test

Load data function call

In [6]:
news, Xy_train, Xy_valid, Xy_test = read_data()

         Date                                               News  \
0  2008-08-08   b"Georgia 'downs two Russian warplanes' as co...   
1  2008-08-11   b'Why wont America and Nato help us? If they ...   

                                    PreProcessedNews  Label  
0  georgia two russian warplane country move brin...      0  
1  wont america nato help wont help help iraq bus...      1  
         Date          Open          High           Low         Close  \
0  2016-07-01  17924.240234  18002.380859  17916.910156  17949.369141   
1  2016-06-30  17712.759766  17930.609375  17711.800781  17929.990234   

      Volume     Adj Close  
0   82160000  17949.369141  
1  133030000  17929.990234  


In [7]:
X_train = Xy_train['PreProcessedNews']
X_valid = Xy_valid['PreProcessedNews']
X_test = Xy_test['PreProcessedNews']
y_train = Xy_train['Label'].to_numpy()
y_valid = Xy_valid['Label'].to_numpy()
y_test = Xy_test['Label'].to_numpy()

Prepare data for vectorization

In [8]:
def prepare_data(train_docs, valid_docs, test_docs, mode):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode training data set
    Xvalid = tokenizer.texts_to_matrix(valid_docs, mode=mode)
    # encode testing data set
    Xtest = tokenizer.texts_to_matrix(test_docs, mode=mode)
    return Xtrain, Xvalid, Xtest

Create vectors for each sentence
* binary
* count
* tfidf
* freq

In [9]:
#modes = ['binary', 'count', 'tfidf', 'freq']

mode = 'binary'

X_train, X_valid, X_test = prepare_data(X_train, X_valid, X_test, mode)

In [10]:
X_valid.shape

(397, 22641)

In [11]:
X_train.shape

(1191, 22641)

In [12]:
X_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 1., 1.]])

Reduce the dimentionality since this is a sparse vector. Use only either one of these. Change the parameter n_components and see.

(1) SVD

In [13]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, n_iter=10)
X_train = svd.fit_transform(X_train)

In [14]:
X_valid = svd.transform(X_valid)
X_test = svd.transform(X_test)

In [15]:
X_train.shape

(1191, 50)

In [16]:
X_valid.shape

(397, 50)

(2) PCA

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca = pca.fit(X_train)

In [18]:
X_train = pca.transform(X_train)
X_valid = pca.transform(X_valid)
X_test = pca.transform(X_test)

In [19]:
X_train.shape

(1191, 50)

In [20]:
X_valid.shape

(397, 50)